## Get Candlestick Data 

In [1]:
import requests
import json
import time
import math
import re
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import yaml
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
now = datetime.now()

In [3]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

### The Configs for Run:

In [4]:
with open ('config.yml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    oanda_api_key = cfg['creds']['oanda_api']
    account_number = cfg['creds']['account_number'] 

### Select the Currency Pair

In [5]:
Load_10K_Records=True

#  'EUR_USD','USD_CAD','EUR_GBP','EUR_AUD','EUR_CHF',
#  'GBP_USD','GBP_CHF','GBP_NZD','GBP_AUD','GBP_CAD',
#  'AUD_CAD','AUD_CHF','AUD_NZD','NZD_USD','EUR_CAD',
#  'USD_CHF','CAD_CHF','NZD_CAD','AUD_USD','EUR_NZD',
#  'NZD_CHF',


currency_pairs = ["USD_CAD"]


timeframe = "H4"
#D #H1 #H4 M30
# https://developer.oanda.com/rest-live-v20/instrument-df/#CandlestickGranularity
price_char = "M"
#M(midpoint candles) #B(bid candles) #A(ask candles) #BA
price_com = "mid"
#mid #bid #ask

# def of OANDA request variable
provider_api_url = 'https://api-fxpractice.oanda.com/v3/accounts/{}/orders'.format(account_number)
request_headers = {
    "Authorization": oanda_api_key,
    "Accept-Datetime-Format": "RFC3339",
    "Connection": "Keep-Alive",
    "Content-Type": "application/json;charset=UTF-8"
}


In [6]:
provider_authorization = 'Bearer {0}'.format(oanda_api_key)

headers = {
    'Content-Type': 'application/json',
    'Authorization': provider_authorization,
}

# Get Candlesticks Data

In [7]:
params_count = (
    ('price', price_char),
    ('count', '5000'),
    ('granularity', timeframe),
)


In [8]:
for pair in currency_pairs:
    first_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair), 
                            headers=headers,
                            params=params_count).json()

In [9]:
if Load_10K_Records:
    datetime_object = parser.parse(first_response['candles'][0]['time'])
    date= datetime_object - relativedelta(years=3)  
    from_date = date.replace(tzinfo=timezone.utc).timestamp()
    params_date = (
        ('count', '5000'),
        ('price', price_char),
        ('from', from_date),
        ('granularity', timeframe),)

    second_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                   headers=headers,
                                   params=params_date).json()
            
    first_response= first_response['candles']  
    second_response= second_response['candles']
    second_response.extend(first_response)
    response=second_response
else:
    response=first_response['candles']

In [10]:
filename = "{}_{}.csv".format(pair, timeframe)
output = []
all_candlesticks = response

for i in range (len(all_candlesticks)):
    result= (convert_date(response[i]['time']))
    output.append([(result[0]),(result[1]),(result[2]),(result[3]),(result[4]),(result[5]),
                    response[i]['time'],
                    response[i]['volume'], 
                    response[i][price_com]['o'],
                    response[i][price_com]['h'],
                    response[i][price_com]['l'],
                    response[i][price_com]['c']])
    
output = pd.DataFrame(output)
output.columns = ['Date','Time','f_time','julian_date','Weekday','Weekday_Name','UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close']
data = output.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [11]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [12]:
data.shape

(9658, 12)

In [13]:
data.head(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
0,2015-10-28,21:00:00,21,301,2,Wednesday,2015-10-28T21:00:00.000000000Z,4683,1.31970,1.32004,1.31863,1.31985
1,2015-10-29,01:00:00,1,302,3,Thursday,2015-10-29T01:00:00.000000000Z,1696,1.31986,1.32142,1.31952,1.32140
2,2015-10-29,05:00:00,5,302,3,Thursday,2015-10-29T05:00:00.000000000Z,4398,1.32138,1.32382,1.32033,1.32244
3,2015-10-29,09:00:00,9,302,3,Thursday,2015-10-29T09:00:00.000000000Z,9021,1.32243,1.32296,1.31908,1.31933
4,2015-10-29,13:00:00,13,302,3,Thursday,2015-10-29T13:00:00.000000000Z,15675,1.31928,1.32053,1.31330,1.31774


In [14]:
data.tail(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
9653,2022-01-07,02:00:00,2,7,4,Friday,2022-01-07T02:00:00.000000000Z,1370,1.27084,1.27237,1.27084,1.27104
9654,2022-01-07,06:00:00,6,7,4,Friday,2022-01-07T06:00:00.000000000Z,3349,1.27106,1.27309,1.27084,1.27138
9655,2022-01-07,10:00:00,10,7,4,Friday,2022-01-07T10:00:00.000000000Z,6584,1.27135,1.27276,1.26792,1.27231
9656,2022-01-07,14:00:00,14,7,4,Friday,2022-01-07T14:00:00.000000000Z,10259,1.27235,1.27265,1.26420,1.26474
9657,2022-01-07,18:00:00,18,7,4,Friday,2022-01-07T18:00:00.000000000Z,3011,1.26476,1.26520,1.26321,1.26429


## Simple Moving Average (SMA)

In [15]:
data['SMA_5'] = data['Close'].rolling(window=5).mean().round(4)
data['SMA_10'] = data['Close'].rolling(window=10).mean().round(4)
data['SMA_20'] = data['Close'].rolling(window=20).mean().round(4)

## Simple Moving Average Range

In [16]:
data['F_SMA_5'] = data['Close'] - data['SMA_5']
data['F_SMA_10'] = data['Close'] - data['SMA_10']
data['F_SMA_20'] = data['Close'] - data['SMA_20']

In [17]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [18]:
data.tail()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,SMA_5,SMA_10,SMA_20,F_SMA_5,F_SMA_10,F_SMA_20
9653,2022-01-07,02:00:00,2,7,4,Friday,2022-01-07T02:00:00.000000000Z,1370,1.27084,1.27237,1.27084,1.27104,1.2725,1.2743,1.2732,-0.00146,-0.00326,-0.00216
9654,2022-01-07,06:00:00,6,7,4,Friday,2022-01-07T06:00:00.000000000Z,3349,1.27106,1.27309,1.27084,1.27138,1.2715,1.2743,1.2730,-0.00012,-0.00292,-0.00162
9655,2022-01-07,10:00:00,10,7,4,Friday,2022-01-07T10:00:00.000000000Z,6584,1.27135,1.27276,1.26792,1.27231,1.2716,1.2740,1.2730,0.00071,-0.00169,-0.00069
9656,2022-01-07,14:00:00,14,7,4,Friday,2022-01-07T14:00:00.000000000Z,10259,1.27235,1.27265,1.26420,1.26474,1.2701,1.2728,1.2725,-0.00536,-0.00806,-0.00776
9657,2022-01-07,18:00:00,18,7,4,Friday,2022-01-07T18:00:00.000000000Z,3011,1.26476,1.26520,1.26321,1.26429,1.2688,1.2714,1.2720,-0.00451,-0.00711,-0.00771


## Price Range

In [19]:
data['O-H'] = data['Open'] - data['High']
data['O-L'] = data['Open'] - data['Low']
data['O-C'] = data['Open'] - data['Close']
data['H-L'] = data['High'] - data['Low']
data['H-C'] = data['High'] - data['Close']
data['L-C'] = data['Low'] - data['Close']

data['Direction'] = data['O-C'].apply(lambda x: 1 if x<0 else 0)

data['col_1'] = data['Open'] - data['Close']

for value in data['col_1']:   
    if value > 0:
        data['col_2'] = data['High'] - data['Open']
        data['col_3'] = data['Close'] - data['Low']
    else:
        data['col_2'] = data['High'] - data['Close']
        data['col_3'] = data['Open'] - data['Low']

#Two Previous Candlesticks 
data['col_4'] = data['col_1'].shift(1)
data['col_5'] = data['col_1'].shift(2)

In [20]:
data = data.dropna()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [21]:
data.tail()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,O-C,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5
9634,2022-01-07,02:00:00,2,7,4,Friday,2022-01-07T02:00:00.000000000Z,1370,1.27084,1.27237,...,-0.00020,0.00153,0.00133,-0.00020,1,-0.00020,0.00153,0.00020,0.00198,-0.00058
9635,2022-01-07,06:00:00,6,7,4,Friday,2022-01-07T06:00:00.000000000Z,3349,1.27106,1.27309,...,-0.00032,0.00225,0.00171,-0.00054,1,-0.00032,0.00203,0.00054,-0.00020,0.00198
9636,2022-01-07,10:00:00,10,7,4,Friday,2022-01-07T10:00:00.000000000Z,6584,1.27135,1.27276,...,-0.00096,0.00484,0.00045,-0.00439,1,-0.00096,0.00141,0.00439,-0.00032,-0.00020
9637,2022-01-07,14:00:00,14,7,4,Friday,2022-01-07T14:00:00.000000000Z,10259,1.27235,1.27265,...,0.00761,0.00845,0.00791,-0.00054,0,0.00761,0.00030,0.00054,-0.00096,-0.00032
9638,2022-01-07,18:00:00,18,7,4,Friday,2022-01-07T18:00:00.000000000Z,3011,1.26476,1.26520,...,0.00047,0.00199,0.00091,-0.00108,0,0.00047,0.00044,0.00108,0.00761,-0.00096


In [22]:
data.shape

(9639, 30)